### Import bibliotek

In [22]:
import requests
from bs4 import BeautifulSoup
import json
import os

Headers json

In [23]:
with open("./headers.json", "r") as f:
    headers = json.load(f)

pętla - kroki 1-4 z readme

In [27]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
        for review in reviews:
            try:
              single_review = {
              "review_id": review['data-entry-id'],
              "author": review.select_one("span.user-post__author-name").text.strip(),
              "recomendation": review.select_one("span.user-post__author-recomendation > em").text.strip(),
              "stars": review.select_one("span.user-post__score-count").text.strip(),
              "content": review.select_one("div.user-post__text").text.strip(),
              "pros": [p.text.strip() for p in review.select("div.review-feature__item--positive")],
              "cons": [c.text.strip() for c in review.select("div.review-feature__item--negative")],
              "likes": review.select_one("button.vote-yes > span").text.strip(),
              "dislikes": review.select_one("button.vote-no > span").text.strip(),
              "publish_date": review.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
              "purchase_date": review.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip(),
              }
              all_reviews.append(single_review)
            except (AttributeError, TypeError):
                pass
        try:
           next_page = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
          next_page = None
        print(all_reviews)


        

   






https://www.ceneo.pl/84514582#tab=reviews
10
[{'review_id': '13551598', 'author': 'b...a', 'recomendation': 'Polecam', 'stars': '4/5', 'content': 'Wykonana dobrze, trochę za duża, z ledwością mieści mi się na biurku, ale może być. Drukuje ciszej w porównaniu z poprzednią drukarką HP 209a. Napełniłam tusze do pełna i po ok.10 kartkach ubyło mi 4 mm czarnego tuszu w pojemniku. Mam nadzieję, że będzie wydajna. Na razie nie mogę tego stwierdzić. Fajnie, że można zobaczyć, ile ubyło tuszu. Dolewanie jest banalnie proste, tusz nie wylewa się. Martwi mnie tylko, że przenosić trzeba ją w pozycji poziomej, bo inaczej tusz może się wylewać. Ogólnie jestem zadowolona z zakupu.', 'pros': ['czyste napełnianie atramentem', 'głośność pracy'], 'cons': [], 'likes': '3', 'dislikes': '3', 'publish_date': '2020-12-10 19:41:04', 'purchase_date': '2020-12-04 21:02:01'}, {'review_id': '18299406', 'author': 'U...o', 'recomendation': 'Polecam', 'stars': '4,5/5', 'content': 'Po 1 dniu uzytkowania mozna stwierdz

Zapisanie wszystkich opinii w bazie danych

In [25]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [28]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as f:
    json.dump(all_reviews,f,indent=4,ensure_ascii=False)